In [49]:
import numpy as np

def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
#         if step % 500 == 0:
#             print(step)
        
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
            
#         eR = np.dot(P,Q)
        e = 0
#         e_loss = 0
#         e_reg = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
#                     e_loss += pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
#                         e_reg += (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
                    
#         if step % 500 == 0:
#             print("Iter {0}: LOSS: {1:.6f}, REG: {2:.6f}".format(step / 50, e_loss, e_reg))
            
        if e < 0.001:
            break
            
    return P, Q.T


In [50]:
def fast_matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    r_plus_mask = np.where(R > 0)
    
    eij = R - np.dot(P, Q.T)
    eij[R <= 0] = 0
    r_plus_bin = np.sign(eij)
    
    PK_mask = np.repeat(np.sum(r_plus_bin, axis=1, keepdims=True), K, axis=1)
    QK_mask = np.repeat(np.sum(r_plus_bin, axis=0, keepdims=True), K, axis=0)
    
    for step in range(steps):
#         if step % 500 == 0:
#             print(step)
            
        eij = R - np.dot(P, Q.T)
        eij[R <= 0] = 0
                
        P += 2 * alpha * np.dot(eij, Q) - alpha * beta * np.multiply(P, PK_mask)
        Q += 2 * alpha * np.dot(eij.T, P) - alpha * beta * np.multiply(Q.T, QK_mask).T

#         eR = np.dot(P, Q.T)
        e_loss = np.sum(np.power(R - np.dot(P, Q.T), 2)[r_plus_mask])
        e_reg = (beta/2) * (np.sum(np.multiply(np.power(P, 2), PK_mask)) + np.sum(np.multiply(np.power(Q.T, 2), QK_mask)))
#         e = e_loss + e_reg

#         if step % 500 == 0:
#             print(e_reg)
#             print("Iter {0}: LOSS: {1:.6f}, REG: {2:.6f}".format(step / 50, e_loss, e_reg))

        if e_loss + e_reg < 0.001:
            break
    
    return P, Q

In [51]:
# import cupy as cp
# import time

In [52]:
R = [
     [5,3,0,1],
     [4,0,0,1],
     [1,1,0,5],
     [1,0,0,4],
     [0,1,5,4],
    ]

R = np.array(R)

M = len(R)
N = len(R[0])
K = 2

_P = np.random.rand(M,K)
_Q = np.random.rand(N,K)


In [53]:
P1 = _P.copy()
P2 = _P.copy()
P3 = _P.copy()
Q1 = _Q.copy()
Q2 = _Q.copy()
Q3 = _Q.copy()

In [55]:
import time


start_time = time.time()
nP, nQ = matrix_factorization(R, P1, Q1, K)
nR1 = np.dot(nP, nQ.T)
end_time = time.time() - start_time
print(nR1)
print("Base Line:", end_time)

start_time = time.time()
nP, nQ = fast_matrix_factorization(R, P2, Q2, K)
nR2 = np.dot(nP, nQ.T)
end_time = time.time() - start_time
print(nR2)
print("Fast:", end_time)

# start_time = time.time()
# R_gpu = cp.asarray(R)
# P3_gpu = cp.asarray(P3)
# Q3_gpu = cp.asarray(Q3)
# print(R_gpu.device)
# start_time = time.time()
# nP, nQ = faster_matrix_factorization(R_gpu, P3_gpu, Q3_gpu, K)
# nR3_gpu = cp.dot(nP, nQ.T)
# end_time = time.time() - start_time
# nR3 = cp.asnumpy(nR3_gpu)

# print(nR3)
# print("Cupy:", end_time)

print(np.sum(np.abs(nR1 - nR2)) / (M*N))


[[ 4.98271884  2.96292742  5.85197762  1.00123113]
 [ 3.97458508  2.37476721  4.83646594  0.99932748]
 [ 1.04381908  0.88906911  5.22158016  4.96841378]
 [ 0.97735461  0.79441097  4.32263155  3.97741342]
 [ 1.56639949  1.13990182  4.94328278  4.01103349]]
Base Line: 1.0313527584075928
[[ 4.98764184  2.99513422  5.36737517  1.00036683]
 [ 3.99684358  2.41604851  4.49684213  1.0009248 ]
 [ 1.00544727  0.98607521  5.78742278  4.9932137 ]
 [ 0.99574565  0.90003221  4.78963641  3.98584978]
 [ 1.1933494   1.01726036  4.9846209   4.00749589]]
Fast: 0.18752002716064453
0.139669935988
